# Flight assignment

A small airlane company produces a weekly plan for the assignment of pilots and co-pilots to flights.
In a pre-processing phase, the list of flights has been analysed to identify the flights that cannot have the same crew members (e.g., flights departing at the same hour). 

Our company follows several rules:
- each flight needs a pilot and a co-pilot to depart. 
- both pilots and co-pilots cannot exceed a weekly amount of flight hours.
- each pilot (resp. co-pilot) is paid based on several factors (e.g., the experience, the rank, flight hours)

The airplane company what to produce the weekly plan minimizing flight costs.

## Data

$P:$ set of pilots  
$C:$ set of co-pilots  
$V:$ set of flights  
$I_v \subseteq V :$ flights incompatibilities of flight $v \in V$  
$f_{pv}:$ cost per hour of pilot $p\in P$ if assigned to flight $v \in V$  
$g_{cv}:$ cost per hour of co-pilot $c\in C$ if assigned to flight $v \in V$  
$q:$ weekly amount of flight hours for each pilot and co-pilot
$t_v:$ flight hours of flight $v\in V$

## Variables

$x_{pv}:$ $1$ if pilot $p$ is assigned to flight $v$, and $0$ otherwise  
$y_{cv}:$ $1$ if co-pilot $c$ is assigned to flight $v$, and $0$ otherwise  

## Objective function

Minimize flight costs
$$z = \min \sum_{v \in V} t_v \cdot ( \sum_{p \in P} f_{pv} \cdot x_{pv} + \sum_{c \in C} g_{cv} \cdot y_{cv} ) $$

## Constraints

One pilot and one co-pilot for each flight
$$ \sum_{p \in P} x_{pv} = 1, \forall v \in V $$  
$$ \sum_{c \in C} y_{cv} = 1, \forall v \in V $$  

Do not exceed flight hours for each pilot and co-pilot
$$ \sum_{v \in V} t_v \cdot x_{pv} \le q, \forall p \in P $$  
$$ \sum_{v \in V} t_v \cdot y_{cv} \le q, \forall c \in C $$

Avoid to assign the same crew to two incompatible flights
$$ x_{pv} + x_{pv'} \le 1, \forall p\in P, v \in V, v' \in I_v $$
$$ y_{cv} + y_{cv'} \le 1, \forall c\in C, v \in V, v' \in I_v $$

## Implementation

Data are stored into three `csv` files, one for the crew members, one for the flights, and one for the costs.

In [ ]:
import pandas as pd

crew = pd.read_csv("crew.csv")
flights = pd.read_csv("flights.csv")
costs = pd.read_csv("costs.csv", index_col=0)

In [ ]:
crew[:2]

In [ ]:
flights[:2]

In [ ]:
costs[:2]

We create the abstract model

In [ ]:
import pyomo.environ as pyopt

model = pyopt.AbstractModel("Flight assignment")

and the sets of pilots, copilots, and flights.

In [ ]:
import math

model.P = pyopt.Set(initialize=crew['Pilots'].dropna().tolist())
model.C = pyopt.Set(initialize=crew['Co-pilots'].dropna().tolist())
model.V = pyopt.Set(initialize=flights['Flights'].tolist())

We then create all the paprameters reading the flight hours for each flight, the incompatibilities between flights, and by setting an arbitrary number of maximum flight hours. 

In [ ]:
model.t = pyopt.Param(model.V, initialize={ r['Flights']: r['Time'] for i, r in flights.iterrows()})
model.q = pyopt.Param(initialize=10)

def read_incompatibilities(model):
    return {(v1,v2): (1 if v1 in flights[v2].tolist() else 0) for v1 in model.V for v2 in model.V}

model.I = pyopt.Param(model.V, model.V, initialize=read_incompatibilities)

To read the incompatibilities we used a function that will run only when creating the instance: the abstract model is instantiated only when method `create_instance()` is called. Before that, no set of our model has been initialized, and we cannot access their elements.

In the same way we use an additional function to read the cost parameters:

In [ ]:
def read_costs(model, p, v):
    return costs.at[p, v]

model.f = pyopt.Param(model.P, model.V, rule=read_costs)
model.g = pyopt.Param(model.C, model.V, rule=read_costs)

All variables are binary ones:

In [ ]:
model.x = pyopt.Var(model.P, model.V, within=pyopt.Binary)
model.y = pyopt.Var(model.C, model.V, within=pyopt.Binary)

The goal is to minimize the flight costs by mininimizing the cost of pilot and co-pilot:

In [ ]:
def objective_function(model):
    return pyopt.sum(model.t[v] * \
                (pyopt.sum(model.f[p,v] * model.x[p,v] for p in model.P) + \
                 pyopt.sum(model.g[c,v] * model.y[c,v] for c in model.C )) for v in model.V)

model.z = pyopt.Objective(rule=objective_function, sense=pyopt.minimize)

We force to select a pilot and co-pilot for each flight:

In [ ]:
def pilot_assignment_cons(model, v):
    return pyopt.sum(model.x[p,v] for p in model.P) == 1

def copilot_assignment_cons(model, v):
    return pyopt.sum(model.y[c,v] for c in model.C) == 1

model.pilot_assignment = pyopt.Constraint(model.V, rule=pilot_assignment_cons)
model.copilot_assignment = pyopt.Constraint(model.V, rule=copilot_assignment_cons)

We forbid a crew member to be on two incompatible flights:

In [ ]:
def pilot_incompatibility_cons(model, p, v1, v2):
    return (model.x[p,v1] + model.x[p,v2] <= 2 - model.I[v1,v2])

def copilot_incompatibility_cons(model, c, v1, v2):
    return (model.y[c,v1] + model.y[c,v2] <= 2 - model.I[v1,v2])

model.pilot_incompatibilities = pyopt.Constraint(model.P, model.V, model.V, rule=pilot_incompatibility_cons)
model.copilot_incompatibilities = pyopt.Constraint(model.C, model.V, model.V, rule=copilot_incompatibility_cons)

We do not exceet pilots and co-pilots flight hours:

In [ ]:
def pilot_hours_cons(model, p):
    return pyopt.sum(model.t[v] * model.x[p,v] for v in model.V) <= model.q

def copilot_hours_cons(model, c):
    return pyopt.sum(model.t[v] * model.y[c,v] for v in model.V) <= model.q

model.pilot_hours = pyopt.Constraint(model.P, rule=pilot_hours_cons)
model.copilot_hours = pyopt.Constraint(model.C, rule=copilot_hours_cons)

All the above constraints are created using additional functions, so that we can easily access to the elements of the sets.

Once the abstract model is create, we also create the instance. 
Then the problem is solved and the total cost printed:

In [ ]:
instance = model.create_instance()
solver = pyopt.SolverFactory('glpk')
results = solver.solve(instance)
print("Weekly plan cost %f" % (pyopt.value(instance.z)))

We finally print the crew member assigned to each flight:

In [ ]:
pd.DataFrame.from_items(
    [(v, [p,c]) for p in instance.P for c in instance.C for v in instance.V 
                if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
      orient='index', columns=['Pilot', 'Co-pilot'])

## Workers go on strike!

Labor unions declared a last minute strike, and some of the pilots and co-pilots of our company joined the strike.

The previous plan must be then updated to recover from the infeasibility, but since all pilots have already received their flight plan, we want them to still flight their previously assigned flights.

The list of pilots and co-pilots on strike is given on a `csv` file:

In [ ]:
strike = pd.read_csv("strike.csv")
strike[:2]

We can use the same model and impose:
  1) to avoid the assignment of flights to pilots on strike
  2) to fix previously generated assignments in such a way that they are not changed
  
This can be done by fixing variables:

In [ ]:
pilots_on_strike = strike['Pilots'].tolist()
for p, v in instance.x:
    if p in pilots_on_strike:
        instance.x[p,v].value = 0
        instance.x[p,v].fixed = True
            
for c, v in instance.y:
    if c in pilots_on_strike:
        instance.y[c,v].value = 0
        instance.y[c,v].fixed = True

for p, v in instance.x:
    if instance.x[p,v].value > 0:
        instance.x[p,v].fixed = True
            
for c, v in instance.y:
    if instance.y[c,v].value > 0:
        instance.y[c,v].fixed = True

If a pilot is on strike, then all its associated variables will be set to $0$, in such a way that the solver will not assign to him any flight. Instead, if any other pilot had already an assigned flight, we fix the corresponding variable so that it is not changed.

We then solve again the problem and provide the updated total cost and the new plan:

In [ ]:
results = solver.solve(instance)
print("Weekly plan cost %f" % (pyopt.value(instance.z)))

In [ ]:
pd.DataFrame.from_items(
    [(v, [p,c]) for p in instance.P for c in instance.C for v in instance.V 
                if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
      orient='index', columns=['Pilot', 'Co-pilot'])